### Initialize

In [9]:
from datetime import datetime
import json
import os
import sys

from ipywidgets import Dropdown
import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0]))
from tools import print_response, StopExecution, get_base_url, get_root_path
import esh_objects as esh

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]

def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}Example/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
            df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


### Search for last name "Kennedy"
-> Simple search for a person with the last name. The results show all persons with the last name "Kennedy"

In [10]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property='lastName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kennedy'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND lastName:Kennedy'))
Total count: 2


,First Name,Last Name,Age
0,Joseph,Kennedy,36


### Search for first name "Alex"
--> Simple search for a person with the first name. The results show all persons with the first name "Alex".

In [11]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property='firstName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Alex'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND firstName:Alex'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


### Serach for age "37" AND first name "Robert" 
--> Simple search for person with two conditions: Condition 1: Age=37 and Condition 2: First name = Robert. Just in case of the two conditions are true then the results will show something.

In [12]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value=37)),
            esh.Comparison(
                property=esh.Property(property='firstName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Jerry'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND age:37 AND firstName:Jerry'))
Total count: 1


,First Name,Last Name,Age
0,Jerry,Roberts,37


## Search for cases
--> Simple search for GUID numbers

In [13]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Cases'),
            esh.Comparison(
                property=esh.Property(property='entityGuid'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='GUIDC00001'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
# execute_search_query(so.dict(exclude_none=True))
r = requests.post(f'{base_url}/v1/query/{selected}Example/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:

        df = pd.DataFrame.from_dict(res['value'])[
            ['entityDesc', 'entityTime', 'status', 'priority','caseStartDate','caseEndDate']]
        df.rename(columns={'entityDesc': 'Description',
                  'entityTime': 'Time', 'status': 'Status', 'priority': 'Prio', 'caseStartDate':'Start Date',
                  'End Date':'End Date'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())



ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Cases AND entityGuid:GUIDC00001'))
Total count: 1


,Description,Time,Status,Prio,Start Date,caseEndDate
0,Kennedy A: Money Laundering,2012-02-05 13:30:00,008,2,2012-02-05,2012-12-20


## Search for person who are under 30 years old
--> Simple search with a special condition statement "Lower than". It is searched for persons who are younger than 30 years old. It is also possible to use "Greater than", "Lower Equals", "Greater Equals".

In [14]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.LessThanOrEqualCaseInsensitive,
                value=esh.StringValue(value=30))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))



ESH query statement: /$all?$top=1&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND age:LE:30'))
Total count: 1


,First Name,Last Name,Age
0,Phillipe,Lomez,28


## Search for names with wildcard *
--> The wildcard serach is used e.g. to use incomplete words 

In [20]:
so = esh.EshObject(
    count=True,
    top=3,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='lastName'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Kenne*'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))




ESH query statement: /$all?$top=3&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND lastName:Kenne*'))
Total count: 2


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Alex,Kennedy,57


## Search for date of birth with ranges between 19750101 - 19801231
--> This simple search shows how ranges can be requested. Here it is searched for a person which is born between 1975 and 1980.

In [23]:
so = esh.EshObject(
    count=True,
    # top=3,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='dob'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[19750101 19801231]'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND dob:[19750101 19801231]'))
Total count: 3


,First Name,Last Name,Age
0,Jerry,Roberts,37
1,Rosario,Gomez,42
2,Rogerio,Gomez,42


## Search for persons who are between 30 and 40 and gender=male
--> Simple search for persons with two conditions: It is shown the ranges of the age 30-40 and the gender male

In [25]:
so = esh.EshObject(
    count=True,
    # top=3,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[30 40]')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male'))
                        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))




ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND age:[30 40] AND gender:male'))
Total count: 2


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Jerry,Roberts,37


## Search for persons where height is between 170 and 180, gender=Male, age between 50-60, country=UK
--> Simple search with several/multiple conditions

In [27]:
so = esh.EshObject(
    count=True,
    # top=3,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='height'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[170 180]')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male')),
            esh.Comparison(
                 property=esh.Property(property='age'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='[50 60]')),
            esh.Comparison(
                 property=esh.Property(property='country'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='UK'))    
                        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))




ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND height:[170 180] AND gender:male AND age:[50 60] AND country:UK'))
Total count: 1


,First Name,Last Name,Age
0,Alex,Kennedy,57


## Search for person who is a male, ethnicity=AFR, and complexion=Dark

--> Search for persons with multiple conditions

In [28]:
so = esh.EshObject(
    count=True,
    # top=3,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(

                property=esh.Property(property='ethnicity'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='AFR')),
            esh.Comparison(
                property=esh.Property(property='gender'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='male')),
            esh.Comparison(
                 property=esh.Property(property='complexion'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Dark')) 
                        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))




ESH query statement: /$all?$top=10&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND ethnicity:AFR AND gender:male AND complexion:Dark'))
Total count: 1


,First Name,Last Name,Age
0,Jerry,Roberts,37
